In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pickle

# 1. Load the data
df = pd.read_csv('test.csv')

# 2. Clean Labels (handling notes in the CSV)
df['label'] = df['label'].astype(str).str[0].astype(int)

# 3. SELECT SENSORS: Heart Rate, Temperature, and SpO2
features = ['heart_rate', 'temperature', 'spo2']
X = df[features]
y = df['label']

# 4. Scale the sensor data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. Recreate the ML Brain
model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model.fit(X_scaled, y)

# 6. Save the final files
with open('nivra_brain.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('nivra_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✅ ML RECREATED: Only Heart Rate, Temperature, and SpO2 are used.")

✅ ML RECREATED: Only Heart Rate, Temperature, and SpO2 are used.


In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pickle

# --- STEP 1: CLEANING THE DATA ---
df = pd.read_csv('test.csv')

# Fix labels: Take only the first character (0 or 1) and make it an integer
df['label'] = df['label'].astype(str).str[0].astype(int)

# Ensure sensors are numeric
sensors = ['heart_rate', 'spo2', 'vibration_intensity']
for col in sensors:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop any empty rows
df = df.dropna(subset=sensors + ['label'])

X = df[sensors]
y = df['label']

# --- STEP 2: SCALING & TRAINING ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# We use 'balanced' weights to make the model extra sensitive to seizures
model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model.fit(X_scaled, y)

# Save the brain and scaler
with open('nivra_brain.pkl', 'wb') as f: pickle.dump(model, f)
with open('nivra_scaler.pkl', 'wb') as f: pickle.dump(scaler, f)

print("NIVRA Brain is now Clean, Trained, and Saved!")

# --- STEP 3: THE 3 TESTS ---
def run_nivra_test(hr, spo2, vib, name):
    scaled = scaler.transform([[hr, spo2, vib]])
    prediction = model.predict(scaled)
    status = "🚨 ALARM (Seizure)" if prediction[0] == 1 else "✅ NORMAL"
    print(f"{name}: {status}")

print("\n--- RESULTS ---")
run_nivra_test(90, 98, 0.02, "Test 1 (Resting)")
run_nivra_test(155, 84, 0.45, "Test 2 (Seizure)")
run_nivra_test(170, 85, 0.40, "Test 3 (Extreme Stress)")

NIVRA Brain is now Clean, Trained, and Saved!

--- RESULTS ---
Test 1 (Resting): ✅ NORMAL
Test 2 (Seizure): 🚨 ALARM (Seizure)
Test 3 (Extreme Stress): 🚨 ALARM (Seizure)


C:\Users\osmng\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\osmng\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\osmng\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
# Get the importance of each sensor
importances = model.feature_importances_
sensors = ['Heart Rate', 'SpO2', 'Vibration']

# Print the importance
for sensor, imp in zip(sensors, importances):
    print(f"{sensor} Importance: {imp*100:.2f}%")

# Visualize it
plt.bar(sensors, importances)
plt.title('Which Sensor is Most Important for NIVRA?')
plt.show()

In [18]:
import pickle

# Save the model to your folder as 'nivra_brain.pkl'
with open('nivra_brain.pkl', 'wb') as file:
    pickle.dump(model, file)

# Also save the 'scaler' (this is very important!)
# The model needs sensor data to be scaled exactly like the training data
with open('nivra_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

print("Brain and Scaler saved successfully!")

Brain and Scaler saved successfully!


In [2]:
def nivra_logic_final(hr, temp, spo2):
    # 1. Align data with the new feature list
    input_data = pd.DataFrame([[hr, temp, spo2]], 
                              columns=['heart_rate', 'temperature', 'spo2'])
    
    # 2. Scale and Predict
    scaled = scaler.transform(input_data)
    pred = model.predict(scaled)[0]
    
    # 3. Safety Veto: Healthy Oxygen (>95%) usually means it's not a seizure
    if spo2 >= 95 and pred == 1:
        return "✅ NORMAL (Filtered: Healthy Oxygen)"
    
    return "🚨 ALARM (Seizure Detected)" if pred == 1 else "✅ NORMAL"

# --- RUN VERIFICATION ---
print(f"Resting: {nivra_logic_final(75, 36.6, 98)}")
print(f"Seizure: {nivra_logic_final(150, 37.8, 85)}")
print(f"High Fever (No Seizure): {nivra_logic_final(100, 39.2, 97)}")

Resting: ✅ NORMAL
Seizure: 🚨 ALARM (Seizure Detected)
High Fever (No Seizure): ✅ NORMAL (Filtered: Healthy Oxygen)


In [1]:
# 15-minute series of Heart Rate, Temperature, and SpO2
# Format: (Minute, HR, Temp, SpO2, Expected_Result)
test_series = [
    (1, 72, 36.5, 98, "✅ NORMAL"),   # Resting
    (2, 75, 36.6, 99, "✅ NORMAL"),   # Resting
    (3, 85, 36.6, 97, "✅ NORMAL"),   # Slight movement
    (4, 110, 36.8, 98, "✅ NORMAL"),  # Walking
    (5, 160, 37.2, 97, "✅ NORMAL"),  # Gym/Running (Should be Vetoed by SpO2)
    (6, 130, 37.0, 96, "✅ NORMAL"),  # Cooling down
    (7, 75, 36.7, 98, "✅ NORMAL"),   # Resting
    (8, 72, 36.6, 99, "✅ NORMAL"),   # Resting
    (9, 80, 36.6, 98, "✅ NORMAL"),   # Resting
    (10, 118, 36.9, 93, "🚨 ALARM"),  # Onset: HR rising, SpO2 dropping
    (11, 145, 37.5, 88, "🚨 ALARM"),  # SEIZURE: High HR, Low SpO2, Temp rising
    (12, 160, 38.0, 84, "🚨 ALARM"),  # PEAK: Extreme symptoms
    (13, 140, 37.8, 86, "🚨 ALARM"),  # Continuing Seizure
    (14, 100, 37.3, 93, "🚨 ALARM"),  # Post-Ictal: Transitioning
    (15, 80, 36.9, 96, "✅ NORMAL")   # Recovery: Back to baseline
]

print(f"{'MIN':<4} | {'HR':<4} | {'TEMP':<5} | {'SpO2':<5} | {'PREDICTION'}")
print("-" * 45)

for m, hr, temp, spo2, expected in test_series:
    # This calls your specific logic function
    prediction = nivra_logic_final(hr, temp, spo2)
    print(f"{m:<4} | {hr:<4} | {temp:<5} | {spo2:<5} | {prediction}")

MIN  | HR   | TEMP  | SpO2  | PREDICTION
---------------------------------------------


NameError: name 'nivra_logic_final' is not defined